# Title

## subTitle



In [1]:
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time

# Visualization
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# Setup environment for running MATLAB code if not done externally
# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

# auto reload modules
%load_ext autoreload
%autoreload 2

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'
output_path: Path = Path(f'{os.getenv("HOME")}/development_psa/models_psa/docs/attachments')

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

date_str: str = '20230703'

filename_process_data = f'{date_str}_solarMED.csv'
# filename_process_data = '20230505_solarMED.csv'

filename_process_data2 = f'{date_str}_MED.csv'

# Resample figures using plotly_resampler
resample_figures = False

# sample_rate = '60s'
sample_rate = '300s'
sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

## Pre-processing

In [2]:
from solarMED_modeling.utils import data_preprocessing, data_conditioning

data_paths = [data_path / filename_process_data, data_path / filename_process_data2]

with open( Path("data") / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric)

2024-05-09 12:22:51.605 | DEBUG    | solarMED_modeling.utils:process_dataframe:67 - Index([], dtype='object')
2024-05-09 12:22:51.606 | INFO     | solarMED_modeling.utils:process_dataframe:74 - Number of duplicate index values in df: 0
2024-05-09 12:22:51.810 | DEBUG    | solarMED_modeling.utils:process_dataframe:67 - Index([], dtype='object')
2024-05-09 12:22:51.811 | INFO     | solarMED_modeling.utils:process_dataframe:74 - Number of duplicate index values in df: 0
2024-05-09 12:22:51.887 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:55 - Signal id FT-SF-003 not found in dataframe columns.
2024-05-09 12:22:51.895 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-05-09 12:22:51.897 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-05-09 12:22:51.898 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-05-09 12:22:51.899 |

## Visualize test data

In [3]:
# df.to_csv('data/dev.csv')

In [6]:
# Update plot config
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)

2024-05-09 12:27:14.021 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:378 - Optimization updates not shown in plot, show_optimization_updates: false
2024-05-09 12:27:14.022 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tamb
2024-05-09 12:27:14.022 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-05-09 12:27:14.024 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False, comparison: False
2024-05-09 12:27:14.026 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tmed_c_in
2024-05-09 12:27:14.026 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-05-09 12:27:14.027 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>m

In [7]:
# Test model
from solarMED_modeling.solar_med import SolarMED

idx_start = 1
span = 1
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    resolution_mode='simple',
    use_models=True,
    use_finite_state_machine=True,
    
    sample_time=sample_rate_numeric,
    
    # If a slow sample time is used, the solar field internal PID needs to be detuned
    # Ki_sf=-0.0001,
    # Kp_sf=-0.005,
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    # cost_w = 3, # €/m³ 
    # cost_e = 0.05, # €/kWhe,
)

# Save model initial state and configuration
model_config = model.model_dump_configuration()
df_mod = model.to_dataframe(df_mod)

model_dump = model.model_dump()

# Run model
# %autoreload 2

for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qhx_s'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        med_vacuum_state=2,
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end} - {df.index[idx]:%H:%M:%S}, elapsed time: {time.time()-start_time:.2f} seconds.")

    df_mod = model.to_dataframe(df_mod)

2024-05-09 12:27:36.114 | INFO     | solarMED_modeling.solar_med:init_matlab_engine:701 - MATLAB engine initialized
2024-05-09 12:27:36.133 | INFO     | solarMED_modeling.solar_med:model_post_init:522 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: simple
            - Sample time: 300.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
        
2024-05-09 12:27:36.145 | DEBUG    | solarMED_modeling.data_validation:within_range_or_zero_or_max:47 - (mmed_s_sp) Value 0.03 is less than 30.00 -> 0.0
2024-05-09 12:27:36.146 | DEBUG    | solarMED_modeling.data_validation:within_range_or_zero_or_max:47 - (mmed_f_sp) Value 0.84 is less than 5.00 -> 0.0
2024-05-09 12:27:36.146 | DEBUG    | so

In [8]:
# Sync model index with measured data
df_mod.index = df.index[idx_start-1:idx if idx<idx_end-1 else idx_end] # idx_start-1 because now we are adding one element after the initialization

# Update plot config
with open( Path("data") / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)

2024-05-09 12:27:49.726 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:378 - Optimization updates not shown in plot, show_optimization_updates: false
2024-05-09 12:27:49.727 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tamb
2024-05-09 12:27:49.728 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-05-09 12:27:49.737 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False, comparison: True
2024-05-09 12:27:49.739 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tmed_c_in
2024-05-09 12:27:49.740 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-05-09 12:27:49.743 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>me

In [7]:
# Save figure
save_figure(
    figure_name=f"SolarMED_validation_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=output_path,
    fig=fig, formats=('png', 'html'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)

2024-05-07 14:00:01.364 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('/home/jmserrano/development_psa/models_psa/docs/attachments')]/SolarMED_validation_20230703.png
2024-05-07 14:00:01.484 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('/home/jmserrano/development_psa/models_psa/docs/attachments')]/SolarMED_validation_20230703.html


In [36]:
from solarMED_modeling import SF_TS_State, MedState
from solarMED_optimization.path_exploration import Node, generate_edges, generate_edges_dataframe
from solarMED_optimization.visualization import plot_state_graph

Np = len(df_mod)

# Generate nodes dataframes
nodes_sfts_df = pd.DataFrame([
    Node(step_idx=step_idx,state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in SF_TS_State]
])
nodes_med_df = pd.DataFrame([
    Node(step_idx=step_idx, state=state).model_dump()
    for step_idx in range(Np) for state in [state for state in MedState]
])

# Generate edges dataframes
edges_list_med = []; edges_list_sfts = []
for step_idx in range(Np):
    edges_list_med = generate_edges(edges_list_med, step_idx, system='MED', Np=Np)
    edges_list_sfts = generate_edges(edges_list_sfts, step_idx, system='SFTS', Np=Np)
    
edges_df_med = generate_edges_dataframe(edges_list_med, nodes_df=nodes_med_df)
edges_df_sfts = generate_edges_dataframe(edges_list_sfts, nodes_df=nodes_sfts_df)


fig = plot_state_graph(
    nodes_df=[nodes_sfts_df, nodes_med_df], 
    system='SolarMED',
    # edges_df=[edges_df_sfts, edges_df_med],
    results_df=df_mod,
    Np=Np, 
    height=800,
    width=1200,
)

fig

2024-05-07 15:16:21.007 | WARNING  | solarMED_optimization.visualization:plot_state_graph:81 - There are more samples than the maximum specified (30), states will be shown every 5 samples. Aliasing may occur


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'RECIRCULATING_TS', 'HEATING_UP_SF', ..., 'RECIRCULATING_TS',
                             'HEATING_UP_SF', 'SF_HEATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '75ac504a-125a-4ea2-a18b-0474e031acfa',
              'x': [0, 0, 0, 0, 5, 5, 5, 5, 10, 10, 10, 10, 15, 15, 15, 15, 20,
                    20, 20, 20, 25, 25, 25, 25, 30, 30, 30, 30, 35, 35, 35, 35, 40,
                    40, 40, 40, 45, 45, 45, 45, 50, 50, 50, 50, 55, 55, 55, 55, 60,
                    60, 60, 60, 65, 65, 65, 65, 70, 70, 70, 70, 75, 75, 75, 75, 80,
                    80, 80, 80, 85, 85, 85, 85, 90, 90, 90, 90, 95, 95, 95, 95,
                    100, 100, 100, 100, 105, 105, 105, 

2024-05-07 14:34:26.351 | WARNING  | solarMED_optimization.visualization:plot_state_graph:81 - There are more samples than the maximum specified (30), states will be shown every 5 samples. Aliasing may occur


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'RECIRCULATING_TS', 'HEATING_UP_SF', ..., 'RECIRCULATING_TS',
                             'HEATING_UP_SF', 'SF_HEATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '7ae95b57-ccf7-421c-94e4-da54eadcc9e7',
              'x': [0, 0, 0, 0, 5, 5, 5, 5, 10, 10, 10, 10, 15, 15, 15, 15, 20,
                    20, 20, 20, 25, 25, 25, 25, 30, 30, 30, 30, 35, 35, 35, 35, 40,
                    40, 40, 40, 45, 45, 45, 45, 50, 50, 50, 50, 55, 55, 55, 55, 60,
                    60, 60, 60, 65, 65, 65, 65, 70, 70, 70, 70, 75, 75, 75, 75, 80,
                    80, 80, 80, 85, 85, 85, 85, 90, 90, 90, 90, 95, 95, 95, 95,
                    100, 100, 100, 100, 105, 105, 105, 

In [ ]:
# Save figure
save_figure(
    figure_name=f"SolarMED_state_evolution_{df.index[0].strftime('%Y%m%d')}", 
    figure_path=output_path,
    fig=fig, formats=('png', 'html', 'svg'), 
    width=fig.layout.width, height=fig.layout.height, scale=2
)